In [ ]:
import tensorflow as tf

base_yolo_model = tf.saved_model.load('yolov5_tf')

input_shape = (640, 640, 3)

inputs = tf.keras.Input(shape=input_shape)

features = base_yolo_model(inputs)

#we use the functional api instead of the sequential one as the yolo model has a complex architecture (isn't sequential) 
x = tf.keras.layers.GlobalAveragePooling2D()(features)
x = tf.keras.layers.Dense(256, activation='relu')(x) 
x = tf.keras.layers.Dropout(0.5)(x) 
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x)#sigmoid cause 2 classes instead of softmax


model = tf.keras.Model(inputs=inputs, outputs=output_layer)

# freeze the base layers for the start of the training process
for layer in base_yolo_model.layers:
    layer.trainable = False

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',  #binary classification
              metrics=['accuracy'])

#bounding boxes handled by yolo ??

In [ ]:
train_img = tf.data.Dataset.list_files("labelled_images/images/*.png")
train_img = train_img.batch(32).prefetch(tf.data.AUTOTUNE)  # define batch size for training efficiency
train_label = tf.data.Dataset.list_files("labelled_images/labels/*.txt")
# probably needs to load to numpy array for compatibility with tensorflow


val_img = tf.data.Dataset.list_files("labelled_images/images/*.png")
val_img = val_img.batch(32).prefetch(tf.data.AUTOTUNE)
val_label = tf.data.Dataset.list_files("labelled_images/labels/*.txt")

In [ ]:
#use early stopping to reduce unecessary training time and risk of overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')

#train a first time
history = model.fit(train_img, train_label,
                    epochs=50,  
                    validation_data=(val_img,val_label)
                    callbacks=[early_stopping])

model.save("trained_yolov5_tf")


In [ ]:
#then unfreeze the yolo layers to fine tune
for layer in base_yolo_model.layers:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # lower learning rate for fine-tuning
              loss='binary_crossentropy',
              metrics=['accuracy'])

#retrain with the first layers weights unfrozen
history2 = model.fit(train_img, train_label,
                    epochs=50,  
                    validation_data=(val_img,val_label)
                    callbacks=[early_stopping])

model.save("fine_tuned_yolov5_tf")
